In [1]:
import numpy as np
import pandas as pd
import math
import pymatgen as mg
from pymatgen.transformations.standard_transformations import RemoveSpeciesTransformation
from pymatgen.transformations.standard_transformations import SupercellTransformation

In [2]:
structure = mg.Structure.from_file("cif/2D/1544657.cif")

cell = [structure.lattice.a, structure.lattice.b, structure.lattice.c]
print(cell)

u = SupercellTransformation(scaling_matrix=((2, 0, 0), (0, 2, 0), (0, 0, 2)))
supercell = u.apply_transformation(structure)

t = RemoveSpeciesTransformation(["H", "C", "N", "S", "O", "I"])
modified_structure = t.apply_transformation(supercell)

print(len(modified_structure))

# Pb-Pb distances in a table

dumb = np.array([modified_structure.get_distance(0, i) for i in range(1, len(modified_structure))])
# around
dumb1 = np.around(dumb,decimals=2)
# remove duplicates
dumb2 = np.array(list(set(dumb1)))
# sort values
dist_lst = np.sort(dumb2)
print(len(dist_lst))
print(dist_lst)

[9.0651, 8.8852, 33.617]
32
13
[ 6.35  8.89  9.07 12.69 17.39 17.41 19.55 19.61 33.62 34.21 34.77 34.82
 35.93]


In [3]:
# Remove distances due to small structural distortions   
disto = 0.5 #Angstrom
def distortions(dist_lst):
    for i in range(0, len(dist_lst)-1):
        a = round(dist_lst[i], 2)
        b = round(dist_lst[i+1], 2)
        if math.isclose(a, b, abs_tol = disto):
            dist_lst[i+1] = dist_lst[i]
    return dist_lst

In [10]:
dumb3 = distortions(dist_lst)
print('distortions',dumb3)
# remove duplicates
dumb4 = np.array(list(set(dumb3)))

# sort values
dist_lst_new = np.sort(dumb4)
print(dist_lst_new)

distortions [ 6.35  8.89  8.89 12.69 17.39 17.39 19.55 19.55 33.62 34.21 34.77 34.77
 35.93]
[ 6.35  8.89 12.69 17.39 19.55 33.62 34.21 34.77 35.93]


In [11]:
# In 2D interlayer spacing always > 2 d_min
# Find d_min = shortest Pb-Pb bond
# Usually around 6.3 in 2D
d_min = min(dst_lst_new)
print(d_min)

dist_lst_min = np.delete(dst_lst_new, np.argwhere((dst_lst_new < d_min*math.sqrt(2)+0.1)))
print('min', dist_lst_min)

# distance always smaller than largerest cell param - d_min
cell_max = max(cell)
dist_lst_crop = np.delete(dist_lst_min, np.argwhere((dist_lst_min >= cell_max)))
print('crop', dist_lst_crop)

6.35
min [33.62 34.21 34.77 35.93 12.69 17.39 19.55]
crop [12.69 17.39 19.55]


In [6]:
# Remove duplicate distances due to short Pb-Pb distances    
def duplicates(dist_lst):
    for i in range(0, len(dist_lst)-1):
        a = round(dist_lst[i]**2, 2)
        b = round(dist_lst[i+1]**2 - d_min**2, 2)
        if math.isclose(a, b, abs_tol = 0.2):
            dist_lst[i+1] = dist_lst[i]
    return dist_lst